<a href="https://colab.research.google.com/github/Vamshi-01/NNDL/blob/main/Assignment2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import math as mt
import random
import pandas as pd

In [3]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/LD.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])

In [5]:
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [7]:
maxX = np.max(data['Previous Day Load'])
minX = np.min(data['Previous Day Load'])
maxY = np.max(data['Present Day Load'])
minY = np.min(data['Present Day Load'])

data['Previous Day Load'] = (data['Previous Day Load'] - minX) / (maxX - minX)
data['Present Day Load'] = (data['Present Day Load'] - minY) / (maxY - minY)

In [8]:
from sklearn.model_selection import train_test_split
x = data['Previous Day Load']
y = data['Present Day Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [9]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
vm = 0
vc = 0
v = 0.5

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        # Calculating updated values of vm and vc
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        # Updating m and c values
        m += vm
        c += vc
print("value of m : ",m)
print("value of c : ",c)

value of m :  0.7171132537325574
value of c :  0.12511078784145221


In [10]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

Training Error :
MAE : 533.2583331279445
MSE : 519633.2481810662
RMSE : 720.8559136062256


In [11]:
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Testing Error :
MAE : 546.2799015481903
MSE : 497098.1953757691
RMSE : 705.0519097029446


In [12]:
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,6161.624640,5671.401261
1,5761.792440,6900.126746
2,7390.270800,6397.771184
3,4597.368840,5845.783699
4,5242.712040,5959.132283
...,...,...
211,6764.319072,7487.549637
212,7390.270800,6726.415010
213,6550.077600,6545.325555
214,5722.043040,5567.554283


In [13]:
x = float(input('give load of present hour on previouus day : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load of present hour :', prediction)

give load of present hour on previouus day : 3021
Predicted load of present hour : 3805.5418763785174
